In [1]:
from utils import BasicDataset, dataSet
from unet import UNet
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.benchmarks.utils import AvalancheDataset, as_classification_dataset
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive,EWC
from avalanche.benchmarks.scenarios.supervised import class_incremental_benchmark
from torch import optim
import numpy as np

In [2]:
train_dataset = BasicDataset(images_dir=r"D:\workdir\CL_for_ImageSegmentation\experience\data\train\img",
                             mask_dir=r"D:\workdir\CL_for_ImageSegmentation\experience\data\train\label")


100%|██████████| 253/253 [00:02<00:00, 100.86it/s]


In [3]:
x = []
y = []
for i in train_dataset:
    x.append(i["x"].tolist())
    y.append(i["targets"].tolist())
train_dataset = dataSet(x,y)

In [41]:
from avalanche.benchmarks import benchmark_from_datasets
from avalanche.benchmarks.utils.data_attribute import DataAttribute
import torch
X_tensor = torch.tensor(x, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.int)

In [64]:
aval_dataset = AvalancheDataset([X_tensor],
                                data_attributes=[DataAttribute(y,"y"),
                                                 DataAttribute([0] * len(train_dataset), "targets_task_labels"),
                                                 ])

In [65]:
bm = benchmark_from_datasets(train=[aval_dataset,aval_dataset,aval_dataset,aval_dataset,aval_dataset],
                             test=[aval_dataset,aval_dataset,aval_dataset,aval_dataset,aval_dataset])

In [66]:
print(f"original training samples = {len(bm.train_stream[0].dataset)}")

original training samples = 253


In [67]:
model = UNet(n_channels=3, n_classes=4)

In [68]:

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

In [69]:
epochs = 5
batch_size= 32
learning_rate = 0.00001
weight_decay = 0.00000001
momentum = 0.999
# CREATE THE STRATEGY INSTANCE (NAIVE)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=4, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

cl_strategy = Naive(
    model, 
    optimizer,
    CrossEntropyLoss(), 
    train_mb_size=batch_size, 
    train_epochs=epochs, 
    eval_mb_size=batch_size,
    evaluator=eval_plugin)

# cl_strategy = EWC(
#     model, 
#     optimizer,
#     CrossEntropyLoss(), 
#     0.01,
#     train_mb_size=batch_size, 
#     train_epochs=epochs, 
#     eval_mb_size=batch_size,
#     evaluator=eval_plugin)


d:\cl_env\lib\site-packages\avalanche\training\templates\base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


In [70]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in bm.train_stream:
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    # results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
-- >> Start of training phase << --
0it [00:00, ?it/s]

ValueError: expected 4D input (got 3D input)

In [73]:
X_tensor[0].shape

torch.Size([3, 447, 513])

In [75]:
model(X_tensor[0:10])

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1174072320 bytes.